## Imports and setting up GDAL environment variables

In [ ]:
import json
import datetime
import requests as rq

import numpy as np

import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.colors import ListedColormap
import matplotlib.patches as mpatches
from IPython.display import Image

import folium

from osgeo import gdal

import shapely
import shapely.wkt
import shapely.geometry
from shapely.geometry import shape, MultiPolygon
from shapely.ops import transform as shapely_transform

import pyproj
from pyproj import Proj

import rasterio as rio
from rasterio.mask import mask
from rasterio.plot import show

from sklearn.cluster import KMeans

gdal.SetConfigOption('GDAL_HTTP_COOKIEFILE', '~/cookies.txt')
gdal.SetConfigOption('GDAL_HTTP_COOKIEJAR', '~/cookies.txt')
gdal.SetConfigOption('GDAL_DISABLE_READDIR_ON_OPEN', 'YES')
gdal.SetConfigOption('CPL_VSIL_CURL_ALLOWED_EXTENSIONS', 'TIF')

mpl.rcParams['figure.dpi'] = 150

## Functions used throughout notebook

- **getndvi** : function to get NDVI array using NIR and RED arrays
- **reproject_aoi** : function takes CRS.from_espg and a WGS Shapely Polygon, to reproject WGS Shapely polygon to desired CRS
- **get_geojson_from_poly** : getting GeoJSON in list (as requried for rasterio mask) from any Shapely Polygon
- **add_geojson_to_map** : Adding a GeoJSON to Folium map

In [ ]:
def getndvi(red, nir):
    return ((nir - red)/(nir + red))

def reproject_aoi(crs, aoishape):
    wgs84 = Proj('+proj=longlat +datum=WGS84 +no_defs', preserve_units=True)
    utm = pyproj.Proj(crs)
    trans_utm = pyproj.Transformer.from_proj(wgs84, utm)
    utm_shapely_polygon = shapely_transform(trans_utm.transform, aoishape)

    return utm_shapely_polygon

def get_geojson_from_poly(shapely_polygon):
    wktext = shapely.wkt.loads(str(shapely_polygon))
    json_str = json.dumps(shapely.geometry.mapping(wktext))

    geometry = [json.loads(json_str)]
    return geometry

def add_geojson_to_map(geojson, map):
    folium.GeoJson(geojson['geometry'],
        name = geojson['properties']['id'],
        zoom_on_click=True,
        style_function=styles).add_to(map)

### Using GET request to download raw Geojson from github gist link

In [ ]:
gist = rq.get('https://gist.githubusercontent.com/thaisbendixen/e126c37a3fa021495414658eeaf86d8d/raw/5d1926dcb3a4b9d631521ba12ea79fdc1ecd2df7/doberitz_multipolygon.geojson')
geoj = gist.json()
geoj

#### Naming polygons within Multipolygon for use later

In [ ]:
## Adding "poly_ID" to the two polygons in the multipolygon geojson for later use

for i in range(len(geoj['features'])):
    geoj['features'][i]['properties']['id'] = f"poly_{i+1}"

In [ ]:
geoj

### adding start and end date using Datetime module, along with Geojson, for querying element84 API


In [ ]:
payload = {}
payload['end_date']=datetime.datetime.today().strftime("%Y-%m-%d")
payload['start_date']=(datetime.datetime.today() - datetime.timedelta(days=20)).strftime("%Y-%m-%d")
payload['aois']=geoj
payload

### Creating Multipolygon in Shapely for geometric operations on the geojson, such as convex_hull, incase that is theinterest to get NDVI values/arrays

In [ ]:
multishape = MultiPolygon([shape(pol['geometry']) for pol in payload['aois']['features']])

In [ ]:
### if the interested area is the overall area covered by both polygons,
### then we get the convex_hull of the multipolygon

outer_coords_shape = multishape.convex_hull
outer_coords_shape

### Visualizing the given Multipolygon with Folium

In [ ]:
#import plugins 
from folium import plugins

# Add custom base maps to folium
basemaps = {
    'Google Maps': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=m&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Maps',
        overlay = True,
        control = True
    ),
    'Google Satellite': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Satellite',
        overlay = True,
        control = True
    ),
    'Google Terrain': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=p&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Terrain',
        overlay = True,
        control = True
    ),
    'Google Satellite Hybrid': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Satellite',
        overlay = True,
        control = True
    )
}


styles =lambda feature: {
        "fillColor": "blue",
        "color": "white",
        "weight": 2,
        "dashArray": "5, 5"}


In [ ]:
m = folium.Map([multishape.centroid.y,multishape.centroid.x], zoom_start=12)

# Add custom basemaps
basemaps['Google Maps'].add_to(m)
basemaps['Google Satellite Hybrid'].add_to(m)

for feature in payload['aois']['features']:
    add_geojson_to_map(feature,m)

# Add a layer control panel to the map.
m.add_child(folium.LayerControl())

#fullscreen
plugins.Fullscreen().add_to(m)
    
m

## Querying AWS/Element84 STAC API endpoint.

- Element8s STAC API endpoint takes start and end date and a bounding box for area of interest.
- The Sentinel 2A images are in the "/sentinel-s2-l2a-cogs/items/" url, which gives links of images kept in the Open Data AWS S3.
- The response is a JSON which has to be sifted through to arrive at ideal images for our usecase


In [ ]:
## Getting overall bounds of the given multipolygon
bounds = list(multishape.bounds)

# removing "spaces" in bounds list/string to avoid issues in sending HTTP requests
bbox = str(bounds).replace(' ','') 
bbox

In [ ]:
## searching for COGs in AWS

url = f"https://earth-search.aws.element84.com/v0/collections/sentinel-s2-l2a-cogs/items? \
limit=550&bbox={bbox}& \
datetime={payload['start_date']}T00:00:00Z/{payload['end_date']}T23:59:59Z"

try:
    search_response = rq.get(url).json()
except Exception as e:
    print(e)
    print('\nSentinel-2 AWS URL failed')

items = search_response['features']

## Initializing an empty list to append only the required data from the STAC JSON response
bandlist = []

## Choosing only dates that have Cloud Cover less than 10%
for item in items:
    if item['properties']['eo:cloud_cover'] < 10:
        image = [
            item['assets']['thumbnail']['href'], # PNG/JPG True Color
            item['properties']['datetime'].split('T')[0],  # DATE in format - 2021-01-01T21:31:13Z 
            item['assets']['B04']['href'],  # RED
            item['assets']['B08']['href'],  # NIR
            item['assets']['visual']['href'], # TIF true color
            item['properties']['sentinel:data_coverage']]
        bandlist.append(image)

print(bandlist)
        
## remove duplicates/double images per day with difference of seconds between each other
## these images are quite common due to the way Sen2 images are stored in AWS, as tiles. 

## function from stackoverflow to iterate through a list
def neighborhood(iterable):
    iterator = iter(iterable)
    prev_item = None
    current_item = next(iterator)  # throws StopIteration if empty.
    for next_item in iterator:
        yield (prev_item, current_item, next_item)
        prev_item = current_item
        current_item = next_item
    yield (prev_item, current_item, None)

to_run = []

## Going through the filtered dates from STAC, looking for duplicated dates,
## then choosing the item that has better [sentinel:data_coverage]

for prev, current, forward in neighborhood(bandlist):
    if prev is not None:
        if not any((prev[1] or current[1]) in sublist for sublist in to_run):
            if prev[1] == current[1]:
                if prev[5] > current[5]:
                    to_run.append(prev)
                else:
                    to_run.append(current)
    else:
        if not any((current[1] or forward[1]) in sublist for sublist in to_run):
            if current[1] == forward[1]:
                if current[5] > forward[5]:
                    to_run.append(current)
                else:
                    to_run.append(forward)

In [ ]:
to_run

In [ ]:
date = to_run[0][1]
b4 = to_run[0][2]
b8 = to_run[0][3]

print('Getting required bands for date :', date, 'from link :', b4)

print('\nOpening red band')

try:
    red = rio.open(b4)
except Exception as e:
    print('\n Issue with Sentinel-2 S3 Link :', e)

print('\nOpening nir band')

nir = rio.open(b8)

## Analysis on Notebook is exploratory, hence there is no loop for the polygons within the multipolygon.

### If required, the overall convex_hull/outer_coordinates of the Multipolygon can be used as well (commented out)

In [ ]:
### Choose individual polygons from multipolygon OR 
### the overall outer coordinates of the multipolygon

# shape_polygon = outer_coords_shape

#################
# for individual polygons out, make GEOMETRY 0 OR 1 ie., first or second polygon
GEOMETRY = 0
#################


shape_polygon = multishape.geoms[GEOMETRY]

In [ ]:
# converting polygon to UTM projection of the image to be queried
# this is used to get the clipped array from S3 Images
utm_shape = reproject_aoi(red.crs, shape_polygon)

In [ ]:
# Using COG links to get clipped arrays from AWS

print('\nClipping NIR band..')
nir_array, nir_transform = mask(
    nir, get_geojson_from_poly(utm_shape), crop=True)
print('NIR array recieved!')

print('\nClipping RED band..')
red_array, red_transform = mask(
    red, get_geojson_from_poly(utm_shape), crop=True)
print('RED array recieved!')

In [ ]:
np.seterr(divide='ignore', invalid='ignore')

# scaling is applied to Sen-2A images to reduce image size and keep arrays as Uint type
# the scaling is multiplied applied back to get ideal NDVI values,
# with float arrays of sen2 bands
SCALE = 0.001

# Getting NDVI array
print('\nComputing NDVI array..')
ndvi = getndvi(red_array[0]*SCALE, nir_array[0]*SCALE)
print('Completed NDVI array!')

if np.count_nonzero(np.isnan(ndvi)) > 0:
    print('\nNAN values in NDVI array, making them 0')
    ndvi = np.nan_to_num(ndvi,0)

# Getting average NDVI
print("\n##################")
print(f"\nMean NDVI for chosen polygon : {payload['aois']['features'][GEOMETRY]['properties']['id']} is = {round(ndvi.mean(),4)}")
print("\n##################")

### Preparing for KMeans

In [ ]:
# Initializing empty array, to later add flattened NDVI for KMeans classfier
flatndvi = np.empty((ndvi.shape[0]*ndvi.shape[1],1))

print('Flattening NDVI array..')
# adding NDVI array flattened
flatndvi[:, 0] = ndvi.flatten()
print('\nFlattened NDVI array!')

In [ ]:
## Applying KMeans clustering to the NDVI raster

NUM_CLASSES = 4

km = KMeans(n_clusters=NUM_CLASSES,random_state=0)
print('Fitting KMeans..')
km.fit(flatndvi)
print('\nPredicting Classes/Clusters..')
km.predict(flatndvi)

print('\nKMeans output ready and reshaped!')
out_dat = km.labels_.reshape((ndvi.shape[0], ndvi.shape[1]))

### Getting True color Geotiff from AWS and comparing the KMeans output

In [ ]:
print('\nGetting RGB image cropped to geometry..')
tci = rio.open(to_run[0][4])
tci_array, tci_transf = mask(tci,get_geojson_from_poly(utm_shape),crop=True)

############
### apply colormap depending on the polygon and the classes output by KMeans

### the output varies in every run of notebook, although the KMeans "random_state" is set to 0
### hence the color scheme needs to be chosen for the outputs manually at the moment.

print('\nChoosing Colormap..')

colors = ["beige", "green","brown","limegreen"]
###########

cmap = ListedColormap(colors)

print('\nDisplaying RGB image and KMeans clusters for comparison!')
show(tci_array)
show(out_dat,cmap=cmap)

## Creating PNG output with matplotlib legends and titles

In [ ]:
plt.imshow(out_dat,cmap=cmap)

brown_patch = mpatches.Patch(color='brown', label='Very Low Vegetation/Water')
green_patch = mpatches.Patch(color='green', label='Dense Vegetation')
lightgreen_patch = mpatches.Patch(color='limegreen', label='Low Vegetation')
white_patch = mpatches.Patch(color='beige', label='Urban/Barren Land')

plt.title(f"Unsupervised Classification for {NUM_CLASSES} Classes - Polygon : {payload['aois']['features'][GEOMETRY]['properties']['id']}\n")
plt.legend(handles=[brown_patch,green_patch,lightgreen_patch,white_patch],bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad=2)
plt.savefig(f"Unsupervised_{payload['aois']['features'][GEOMETRY]['properties']['id']}.png",bbox_inches="tight")
plt.show()